In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import corner
import h0rton.tdlmc_data
import h0rton.tdlmc_utils

%matplotlib inline

## Visualizing the Rung1 truth data

__Author:__ Ji Won Park (@jiwoncpark)
__Created:__ 9/10/2019
__Last run:__ 10/07/2019

__Goals:__
We visualize the Rung 1 data.

__Before_running:__
Download and extract the TDLMC data by running, e.g. at the root of the repo,
```bash
source h0rton/tdlmc_data/download_tdlmc_data.sh
```

The following line converts the TDLMC data into a single dataframe.

In [ ]:
df = h0rton.tdlmc_utils.convert_to_dataframe(rung=1, save_csv_path=None)

Let's additionally compute the offset of the source from the lens center, in arcsec.

In [ ]:
df['src_offset'] = np.array([np.linalg.norm(df['host_pos'].iloc[i]) for i in range(len(df))])

Here are the list of columns available. Note that, for a single rung, the `H0` values will be the same.

In [ ]:
sorted(df.columns.values)

In [ ]:
[df['host_pos'].iloc[i][1] for i in range(len(df))]

In [ ]:
np.mean([df['host_pos'].iloc[i][0] for i in range(len(df))]), np.mean([df['host_pos'].iloc[i][1] for i in range(len(df))])

## Marginal 1D distributions

In [ ]:
np.pi*0.5

In [ ]:
col_name = 'host_mag'
print("Mean: ", df[col_name].mean())
print("Std: ", df[col_name].std())
plt.hist(df[col_name])

## Distance between lens and source

In [ ]:
 df['z_src'] - df['z_lens']

In [ ]:
cols = [
 #'H0',
 #'agn_img_amp',
 #'agn_img_mag',
 #'agn_img_pos_x',
 #'agn_img_pos_y',
 'agn_src_amp',
 'ext_shear_b',
 #'ext_shear_e1',
 #'ext_shear_e2',
 'ext_shear_phi_G',
 #'host_img_mag',
 'host_mag',
 #'host_name',
 #'host_pos',
 'host_r_eff',
 'kappa_ext',
 'lens_light_R_sersic',
 #'lens_light_amp_sersic',
 'lens_light_mag_sersic',
 'lens_light_n_sersic',
 'lens_light_phi_G',
 'lens_light_q',
 #'lens_mass_center_x',
 #'lens_mass_center_y',
 'lens_mass_gamma',
 'lens_mass_phi_G',
 'lens_mass_q',
 'lens_mass_theta_E',
 #'measured_time_delays',
 #'measured_time_delays_err',
 #'measured_vel_disp',
 #'measured_vel_disp_err',
 #'name',
 #'seed_path',
 'src_offset',
 #'td_distance',
 #'time_delays',
 'vel_disp',
 'z_lens',
 'z_src']

n_params = len(cols)

plot = corner.corner(df[cols],
                    color='tab:blue', 
                    smooth1d=None, 
                    labels=cols,
                    show_titles=True,
                    fill_contours=False,
                    plot_contours=False,
                     weights=None,
                    range=[1.0]*n_params,
                     quiet=True,
                    hist_kwargs=dict(density=False, ))

Some correlations are not physical. Take this sub-cornerplot:

In [ ]:
cols = [
 #'H0',
 #'agn_img_amp',
 #'agn_img_mag',
 #'agn_img_pos_x',
 #'agn_img_pos_y',
 'agn_src_amp',
 'ext_shear_b',
 #'ext_shear_e1',
 #'ext_shear_e2',
 #'ext_shear_phi_G',
 #'host_img_mag',
 'host_mag',
 #'host_name',
 #'host_pos',
 #'host_r_eff',
 'kappa_ext',
 #'lens_light_R_sersic',
 #'lens_light_amp_sersic',
 'lens_light_mag_sersic',
 #'lens_light_n_sersic',
 #'lens_light_phi_G',
 #'lens_light_q',
 #'lens_mass_center_x',
 #'lens_mass_center_y',
 'lens_mass_gamma',
 #'lens_mass_phi_G',
 #'lens_mass_q',
 #'lens_mass_theta_E',
 #'measured_time_delays',
 #'measured_time_delays_err',
 #'measured_vel_disp',
 #'measured_vel_disp_err',
 #'name',
 #'seed_path',
 #'src_offset',
 #'td_distance',
 #'time_delays',
 #'vel_disp',
 'z_lens',
 'z_src']

n_params = len(cols)

plot = corner.corner(df[cols],
                    color='tab:blue',
                    smooth1d=None,
                    alpha=1.0,
                    labels=cols,
                    show_titles=True,
                    fill_contours=False,
                    plot_contours=False,
                     weights=None,
                    range=[1.0]*n_params,
                     quiet=True,
                    hist_kwargs=dict(density=False, ))

## View the images

In [ ]:
import h0rton.tdlmc_data as tdlmc_data
from pathlib import Path
from astropy.visualization import MinMaxInterval, AsinhStretch, ImageNormalize

In [ ]:
img_dir = os.path.join(tdlmc_data.__path__[0], 'rung1')
img_files = []
for f in Path(img_dir).rglob('*/psf.fits'):
    img_files.append(f)
n_img = len(img_files)
print(n_img)

In [ ]:
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits

In [ ]:
plt.figure(figsize=(20, 20))
n_columns = 5
bkg_std = []
img_max = []
imgs = []
for i, img_file in enumerate(img_files):
    #image_file = get_pkg_data_filename(img_file)
    orig_img_arr = fits.getdata(img_file, ext=0)
    imgs.append(orig_img_arr)
    bkg_std.append(np.mean(orig_img_arr[:, -20:]))
    img_max.append(np.max(orig_img_arr))
    plt.subplot(n_img/n_columns+1, n_columns, i+1)
    plt.imshow(orig_img_arr, origin='lower')

In [ ]:
plt.hist(bkg_std)

In [ ]:
plt.hist(img_max)

In [ ]:

plt.figure(figsize=(20, 20))
n_columns = 5
for i, img_file in enumerate(img_files):
    #image_file = get_pkg_data_filename(img_file)
    orig_img_arr = fits.getdata(img_file, ext=0)
    norm = ImageNormalize(orig_img_arr, interval=MinMaxInterval(), stretch=AsinhStretch())
    plt.subplot(n_img/n_columns+1, n_columns, i+1)
    plt.imshow(np.log(orig_img_arr), origin='lower')